In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
m

In [3]:
import json
import pickle
from copy import deepcopy
from pathlib import Path

import gurobipy
from gurobipy import GRB
import dgl
import numpy as np
import torch
import matplotlib.pyplot as plt

from src.problem import get_model
from src.dataset import InstanceDataset
from src.net import InstanceGCN
from src.utils import load_from_wandb

In [4]:
instances_dir = Path('../data/raw')
instances_fpaths = list(instances_dir.glob('97_*.json'))

net = InstanceGCN(1, readout_op=None)
net = load_from_wandb(net, 'vty6z1q3', 'sat-gnn', 'model_last')
net.eval()

ds = InstanceDataset(instances_fpaths, split='val', return_model=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-10


<gurobi.Model MIP instance Unnamed: 10163 constrs, 3783 vars, Parameter changes: TimeLimit=60.0, Username=(user-defined), LogToConsole=0>

# Warm start

In [7]:
def get_ws_times_n_vars(graph, model, net, name=None, plot=True, timeout=60):
    vars_names = np.core.defchararray.array([v.getAttr(GRB.Attr.VarName) for v in model.getVars()])
    vars_names = vars_names[(vars_names.find('x(') >= 0) | (vars_names.find('phi(') >= 0)]

    # baseline results
    model_ = model.copy()
    model_.setParam('TimeLimit', timeout)
    model_.update()
    model_.optimize()
    baseline_runtime = model_.Runtime
    baseline_obj = model_.ObjVal
    baseline_gap = model_.MIPGap

    with torch.no_grad():
        x_hat = torch.sigmoid(net(graph)).squeeze(0)

    most_certain_idx  = (x_hat - 0.5).abs().sort(descending=True).indices


    runtimes = list()
    objs = list()
    gaps = list()
    ns = list()
    for n in [0, 50, 100, 200, 500, 1000, len(x_hat)]:
    # for n in [0, 50, 100, 150]:
        if n == 0:
            runtimes.append(baseline_runtime)
            objs.append(baseline_obj)
            gaps.append(baseline_gap)
            ns.append(n)
            continue

        fixed_x_hat = (x_hat[most_certain_idx[:n]] > .5).to(x_hat)
        fixed_vars_names = vars_names[most_certain_idx[:n]]

        # fix variables
        model_ = model.copy()
        for fixed_var_name, fixed_var_X in zip(fixed_vars_names, fixed_x_hat):
            model_.getVarByName(fixed_var_name).Start = fixed_var_X

        model_.setParam('TimeLimit', timeout)
        model_.update()
        model_.optimize()

        if model_.status not in [2, 9]:
            print('warm starting with n=',n,' made the optimizatio terminate with status ',model_.status)
            break

        runtimes.append(model_.Runtime)
        objs.append(model_.ObjVal)
        gaps.append(model_.MIPGap)
        ns.append(n)

    objs = [100 * o / baseline_obj for o in objs]

    if plot:
        fig, ax = plt.subplots()
        ax.plot(ns, runtimes, '.-', label='Runtime')
        # ax.set_xlim((0, x_opt.shape[0]))
        ax.grid()
        ax.legend(loc=1)

        ax.set_title('Early fixing performance '+str(name))
        ax.set_xlabel('# fixed variables')
        ax.set_ylabel('Runtime')

        ax.set_xlim(0,max(ns))

        obj_ax = ax.twinx()
        obj_ax.plot(ns, objs, '.-', c='r', label='Objective')
        obj_ax.legend(loc=4)
        obj_ax.set_ylim(min(objs), 100)
        obj_ax.set_ylabel('Objective')

        fig.show()

    return ns, runtimes, objs, gaps

In [10]:
g, _, m = ds[-1]

ns, runtimes, objs, gaps = get_ws_times_n_vars(g, m, net, timeout=10, plot=False)
ns, runtimes, objs, gaps

([0, 50, 100, 200, 500, 1000, 1746],
 [0.840144157409668,
  0.5113790035247803,
  1.2048890590667725,
  0.6877460479736328,
  0.6777899265289307,
  0.8448271751403809,
  0.6791079044342041],
 [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

# Hinting

In [14]:
m.getVars()[0].VarHintPri

0

In [15]:
def get_ht_times_n_vars(graph, model, net, name=None, plot=True, timeout=60):
    vars_names = np.core.defchararray.array([v.getAttr(GRB.Attr.VarName) for v in model.getVars()])
    vars_names = vars_names[(vars_names.find('x(') >= 0) | (vars_names.find('phi(') >= 0)]

    # baseline results
    model_ = model.copy()
    model_.setParam('TimeLimit', timeout)
    model_.update()
    model_.optimize()
    baseline_runtime = model_.Runtime
    baseline_obj = model_.ObjVal
    baseline_gap = model_.MIPGap

    with torch.no_grad():
        x_hat = torch.sigmoid(net(graph)).squeeze(0)

    most_certain_idx  = (x_hat - 0.5).abs().sort(descending=True).indices


    runtimes = list()
    objs = list()
    gaps = list()
    ns = list()
    for n in [0, 50, 100, 200, 500, 1000, len(x_hat)]:
    # for n in [0, 50, 100, 150]:
        if n == 0:
            runtimes.append(baseline_runtime)
            objs.append(baseline_obj)
            gaps.append(baseline_gap)
            ns.append(n)
            continue

        fixed_x_hat = (x_hat[most_certain_idx[:n]] > .5).to(x_hat)
        certainty_x_hat = (x_hat[most_certain_idx[:n]] - 0.5).abs() * 2
        fixed_vars_names = vars_names[most_certain_idx[:n]]

        # fix variables
        model_ = model.copy()
        for fixed_var_name, fixed_var_X, var_certainty in zip(fixed_vars_names, fixed_x_hat, certainty_x_hat):
            model_.getVarByName(fixed_var_name).VarHintVal = fixed_var_X
            model_.getVarByName(fixed_var_name).VarHintPri = int(var_certainty * 100)

        model_.setParam('TimeLimit', timeout)
        model_.update()
        model_.optimize()

        if model_.status not in [2, 9]:
            print('warm starting with n=',n,' made the optimizatio terminate with status ',model_.status)
            break

        runtimes.append(model_.Runtime)
        objs.append(model_.ObjVal)
        gaps.append(model_.MIPGap)
        ns.append(n)

    objs = [100 * o / baseline_obj for o in objs]

    if plot:
        fig, ax = plt.subplots()
        ax.plot(ns, runtimes, '.-', label='Runtime')
        # ax.set_xlim((0, x_opt.shape[0]))
        ax.grid()
        ax.legend(loc=1)

        ax.set_title('Early fixing performance '+str(name))
        ax.set_xlabel('# fixed variables')
        ax.set_ylabel('Runtime')

        ax.set_xlim(0,max(ns))

        obj_ax = ax.twinx()
        obj_ax.plot(ns, objs, '.-', c='r', label='Objective')
        obj_ax.legend(loc=4)
        obj_ax.set_ylim(min(objs), 100)
        obj_ax.set_ylabel('Objective')

        fig.show()

    return ns, runtimes, objs, gaps

In [19]:
g, _, m = ds[2]

ns, runtimes, objs, gaps = get_ht_times_n_vars(g, m, net, timeout=10, plot=False)
ns, runtimes, objs, gaps

([0, 50, 100, 200, 500, 1000, 4268],
 [10.009244918823242,
  10.007880210876465,
  10.005330085754395,
  10.010838031768799,
  10.027592182159424,
  10.018505096435547,
  10.02341604232788],
 [100.0,
  99.77518904557532,
  99.74453300633559,
  99.64234620886981,
  99.77518904557532,
  99.77518904557532,
  99.77518904557532],
 [0.05773554056815859,
  0.06421548545678,
  0.06669398627189838,
  0.06573684750282022,
  0.06278164686603852,
  0.06411306841458418,
  0.06329373207701762])